In [115]:
#Tested on Julia 1.5.3

using LinearAlgebra

A = [1 0 1 0; 0 1 0 1; 0 0 1 0; 0 0 0 1]
sigma = 0.09774;

#Kalman filtering 

In [116]:
# Example with seven cross-fix points
#true position of target
#note zero #acceleration
truex = [1.4, 1.3, 1.2, 1.1, 1.0, 0.9, 0.8]
truey = [1.65, 1.35, 1.05, 0.75, 0.45, 0.15, -0.15]

#these are the inverse transformation functions F and G
truetheta1 = pi * (truey .< 0) .+ atan.(truex./truey);
truetheta2 = pi * (truey .> 1) .+ atan.(truex./(1 .- truey))

theta1 = (truetheta1 .+ randn(7)*sigma)
theta2 = (truetheta2 .+ randn(7)*sigma)

#here are the observations x and y
#these are also the transformation functions f and g
x = (sin.(theta1) .* sin.(theta2) ) ./ (sin.(theta1 .+ theta2))
y = (cos.(theta1) .* sin.(theta2) ) ./ (sin.(theta1 .+ theta2));


In [117]:
#for any observation after #2, directional velocities 
vx = similar(x)
vx[1] = 0
for i in 2:length(x)
     vx[i] = x[i] - x[i-1]
end

vy = similar(y)
vy[1] = 0
for i in 2:length(y)
     vy[i] = y[i] - y[i-1]
end


In [118]:
#partial derivatives of f and g wrt theta1 and theta2

dfdtheta1 = (1 ./ sin.(theta1 .+ theta2)) .*(sin.(theta2).*cos.(theta1) .- sin.(theta2).*sin.(theta1).*cot.(theta1 .+ theta2))

dfdtheta2 = (1 ./ sin.(theta1 .+ theta2)).*(sin.(theta1).*cos.(theta2) .- sin.(theta2).*sin.(theta1).*cot.(theta1 .+ theta2))

dgdtheta1 = (1 ./ sin.(theta1 .+ theta2)).*(-sin.(theta2).*sin.(theta1) .- sin.(theta2).*cos.(theta1).*cot.(theta1 .+ theta2))

dgdtheta2 = (1 ./ sin.(theta1 .+ theta2)).*(cos.(theta2).*cos.(theta1) .- sin.(theta2).*cos.(theta1).*cot.(theta1 .+ theta2));


In [119]:
#delta method

varX = (sigma ^ 2) .* (dfdtheta1 .^ 2) .+ (sigma ^ 2) .* (dfdtheta2 .^ 2)
varY = (sigma ^ 2) .* (dgdtheta1 .^ 2) .+ (sigma ^ 2) .* (dgdtheta2 .^ 2)

covXY = (sigma ^ 2) .* (dfdtheta1 .* dgdtheta1) .+ (sigma ^ 2) .* (dfdtheta2 .* dgdtheta2);


In [120]:
varVx = similar(varX)
varVx[1] = 0
for i in 2:length(varX)
     varVx[i] = varX[i] + varX[i-1]
end

varVy = similar(varY)
varVy[1] = 0
for i in 2:length(varY)
     varVy[i] = varY[i] + varY[i-1]
end


In [121]:
#Kalman filtering

#initial Kalman state
xk1 = [x[2], y[2], vx[2], vy[2]]

#initial Process Covariance Matrix is initial Sensor Data Error
Pk1 = [varX[2] covXY[2] 0 0; covXY[2] varY[2] 0 0; 0 0 varVx[2] 0; 0 0 0 varVy[2]]

#predicted Kalman state
xkp2 = A*xk1

#predicted Process Covariance Matrix
Pkp2 = A*Pk1*A' 

#setting off-diagonal elements to zero (except 2 and 5)
Pkp2[3] = 0
Pkp2[4] = 0
Pkp2[7] = 0
Pkp2[8] = 0
Pkp2[9] = 0
Pkp2[10] = 0
Pkp2[12] = 0
Pkp2[13] = 0
Pkp2[14] = 0
Pkp2[15] = 0

#Sensor Data Error 
R2 = [varX[3] covXY[3] 0 0; covXY[3] varY[3] 0 0; 0 0 varVx[3] 0; 0 0 0 varVy[3]]

#Kalman Gain
K2 = (Pkp2' + R2)\(Pkp2)

#Observation
yk2 = [x[3], y[3], vx[3], vy[3]]

#Updated Kalman state
xk2 = xkp2 + K2*(yk2 - xkp2);


4-element Array{Float64,1}:
  1.0740088019686453
  0.9490110787273948
  0.1586212988825807
 -0.15559566083422094

In [122]:
#Updated Process Covariance Matrix
Pk2 = (I(4) - K2)*Pkp2*((I(4) - K2)') + K2*R2*K2'

xkp3 = A*xk2
Pkp3 = A*Pk2*A'
Pkp3[3] = 0
Pkp3[4] = 0
Pkp3[7] = 0
Pkp3[8] = 0
Pkp3[9] = 0
Pkp3[10] = 0
Pkp3[12] = 0
Pkp3[13] = 0
Pkp3[14] = 0
Pkp3[15] = 0
R3 = [varX[4] covXY[4] 0 0; covXY[4] varY[4] 0 0; 0 0 varVx[4] 0; 0 0 0 varVy[4]]
K3 = (Pkp3 + R3)\Pkp3
yk3 = [x[4], y[4], vx[4], vy[4]]
xk3 = xkp3 + K3*(yk3 - xkp3);


4-element Array{Float64,1}:
  1.3483160704539383
  0.9052667475043764
  0.16796167124438452
 -0.10688884101743822

In [123]:
Pk3 = (I(4) - K3)*Pkp3*((I(4) - K3)') + K3*R3*K3'

xkp4 = A*xk3
Pkp4 = A*Pk3*A'
Pkp4[3] = 0
Pkp4[4] = 0
Pkp4[7] = 0
Pkp4[8] = 0
Pkp4[9] = 0
Pkp4[10] = 0
Pkp4[12] = 0
Pkp4[13] = 0
Pkp4[14] = 0
Pkp4[15] = 0
R4 = [varX[5] covXY[5] 0 0; covXY[5] varY[5] 0 0; 0 0 varVx[5] 0; 0 0 0 varVy[5]]
K4 = (Pkp4 + R4)\Pkp4
yk4 = [x[5], y[5], vx[5], vy[5]]
xk4 = xkp4 + K4*(yk4 - xkp4);

4-element Array{Float64,1}:
  0.9771039841394796
  0.46228641959907824
 -0.05095464712982317
 -0.27316518339579454

In [124]:
Pk4 = (I(4) - K4)*Pkp4*((I(4) - K4)') + K4*R4*K4'

xkp5 = A*xk4
Pkp5 = A*Pk4*A'
Pkp5[3] = 0
Pkp5[4] = 0
Pkp5[7] = 0
Pkp5[8] = 0
Pkp5[9] = 0
Pkp5[10] = 0
Pkp5[12] = 0
Pkp5[13] = 0
Pkp5[14] = 0
Pkp5[15] = 0
R5 = [varX[6] covXY[6] 0 0; covXY[6] varY[6] 0 0; 0 0 varVx[6] 0; 0 0 0 varVy[6]]
K5 = (Pkp5 + R5)\Pkp5
yk5 = [x[6], y[6], vx[6], vy[6]]
xk5 = xkp5 + K5*(yk5 - xkp5);

4-element Array{Float64,1}:
  0.9417601243167208
  0.19369109897963482
 -0.0032311645622973145
 -0.2571994625510113

In [132]:
Pk5 = (I(4) - K5)*Pkp5*((I(4) - K5)') + K5*R5*K5'

xkp6 = A*xk5
Pkp6 = A*Pk5*A'
Pkp6[3] = 0
Pkp6[4] = 0
Pkp6[7] = 0
Pkp6[8] = 0
Pkp6[9] = 0
Pkp6[10] = 0
Pkp6[12] = 0
Pkp6[13] = 0
Pkp6[14] = 0
Pkp6[15] = 0
R6 = [varX[7] covXY[7] 0 0; covXY[7] varY[7] 0 0; 0 0 varVx[7] 0; 0 0 0 varVy[7]]
K6 = (Pkp6 + R6)\Pkp6
yk6 = [x[7], y[7], vx[7], vy[7]]
xk6 = xkp6 + K6*(yk6 - xkp6);

4-element Array{Float64,1}:
  1.0906555189134495
  0.08746654845492213
  0.04579056152579714
 -0.2829495003481493

In [126]:
Pk6 = (I(4) - K6)*Pkp6*((I(4) - K6)') + K6*R6*K6'

#How to draw 95% confidence error ellipses from 
#a Covariance Matrix?
#drawing two of them, one uses top-left 2x2 submatrix
#of R6: the Blue Ellipse
#the other uses top-left 2x2 submatrix of Pk6: the Green Ellipse

trkal = Pk6[1] + Pk6[6]
detkal = Pk6[1] * Pk6[6] - Pk6[2]^2

lambdakal = (trkal + (trkal^2 - 4*detkal)^(0.5))/2

trdelta = R6[1] + R6[6]
detdelta = R6[1] * R6[6] - R6[2]^2
lambdadelta = (trdelta + (trdelta^2 - 4*detdelta)^(0.5))/2

#usually the Dkal is less than Ddelta
#which means Kalman confidence ellipse
#has a smaller major axis than the Observation confidence ellipse
#DKaltrue < Ddeltatrue often happens but is more a matter of chance
#and it means the Kalman state position is closer to the true 
#position than the Observation
#Dkaltrue < Dkal is when the Kalman state position is 
#within a distance equal to the major axis of
#the Kalman Error Confidence Ellipse to the true position of the target

@show Ddelta = (5.991*lambdadelta)^0.5
@show Ddeltatrue = ((x[7] - truex[7])^2 + (y[7] - truey[7])^2)^0.5
@show Dkal = (5.991*lambdakal)^0.5
@show Dkaltrue = ((xk6[1] - truex[7])^2 + (xk6[2] - truey[7])^2)^0.5
@show Dkal < Ddelta
@show Dkaltrue < Ddeltatrue
@show Dkaltrue < Dkal

Ddelta = (5.991lambdadelta) ^ 0.5 = 1.289838839393829
Ddeltatrue = ((x[7] - truex[7]) ^ 2 + (y[7] - truey[7]) ^ 2) ^ 0.5 = 0.7523764114866829
Dkal = (5.991lambdakal) ^ 0.5 = 0.5281294738823917
Dkaltrue = ((xk6[1] - truex[7]) ^ 2 + (xk6[2] - truey[7]) ^ 2) ^ 0.5 = 0.3753278464355402
Dkal < Ddelta = true
Dkaltrue < Ddeltatrue = true
Dkaltrue < Dkal = true


true

In [134]:
@show Aobs = [x[1], y[1]]
@show Bobs = [x[2], y[2]]
@show Cobs = [x[3], y[3]]
@show Dobs = [x[4], y[4]]
@show Eobs = [x[5], y[5]]
@show Fobs = [x[6], y[6]]
@show Gobs = [x[7], y[7]]
@show Ckal = xk2[1:2]
@show Dkal = xk3[1:2]
@show Ekal = xk4[1:2]
@show Fkal = xk5[1:2]
@show Gkal = xk6[1:2];


Aobs = [x[1], y[1]] = [1.338001378912692, 1.8414158596986516]
Bobs = [x[2], y[2]] = [0.8581576559764524, 1.1306583304872226]
Cobs = [x[3], y[3]] = [1.2580464962516391, 1.0413365903142222]
Dobs = [x[4], y[4]] = [1.4474211997064996, 1.00947389262003]
Eobs = [x[5], y[5]] = [0.8991184436010751, 0.4273487860554108]
Fobs = [x[6], y[6]] = [0.9489529361231394, 0.193172677360586]
Gobs = [x[7], y[7]] = [1.5046656222848585, -0.4136600563822128]
Ckal = xk2[1:2] = [1.0740088019686453, 0.9490110787273948]
Dkal = xk3[1:2] = [1.3483160704539383, 0.9052667475043764]
Ekal = xk4[1:2] = [0.9771039841394796, 0.46228641959907824]
Fkal = xk5[1:2] = [0.9417601243167208, 0.19369109897963482]
Gkal = xk6[1:2] = [1.0906555189134495, 0.08746654845492213]


In [133]:
R6

4×4 Array{Float64,2}:
  0.230214  -0.100939   0.0       0.0
 -0.100939   0.0631216  0.0       0.0
  0.0        0.0        0.261612  0.0
  0.0        0.0        0.0       0.0701483

In [129]:
@show theta1ex = copy(theta1);
@show theta2ex = copy(theta2);
@show xex = copy(x);
@show yex = copy(y);
#values in example 2 in Annex are:
#theta1ex = [0.6284, 0.6492, 0.8794, 0.9618, 1.1271, 1.3700, 1.8391];
#theta2ex = [2.1322, 1.7219, 1.6036, 1.5773, 1.0037, 0.8662, 0.8166];
#xex = [1.338, 0.858, 1.258, 1.447, 0.899, 0.949, 1.505];
#yex = [1.841, 1.131, 1.041, 1.009, 0.427, 0.193, -0.414];

theta1ex = copy(theta1) = [0.6283663159064792, 0.649229526275132, 0.8793675693164166, 0.9617977774710615, 1.127105250944439, 1.36997618227606, 1.8390865440876325]
theta2ex = copy(theta2) = [2.132166725697592, 1.7218903927059386, 1.6036422700559108, 1.5773415931850534, 1.0036833189898502, 0.8661693210542865, 0.8165722315413557]
xex = copy(x) = [1.338001378912692, 0.8581576559764524, 1.2580464962516391, 1.4474211997064996, 0.8991184436010751, 0.9489529361231394, 1.5046656222848585]
yex = copy(y) = [1.8414158596986516, 1.1306583304872226, 1.0413365903142222, 1.00947389262003, 0.4273487860554108, 0.193172677360586, -0.4136600563822128]


In [130]:
Pk6

4×4 Array{Float64,2}:
 0.0401718   0.00643297  0.0        0.0
 0.00643297  0.0400752   0.0        0.0
 0.0         0.0         0.0229445  0.0
 0.0         0.0         0.0        0.00516633